In [69]:
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader,random_split
import torch.nn.functional as F
import torchvision.transforms as transforms

In [70]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
])

dataset = datasets.ImageFolder("./drive/MyDrive/Breast Cancer dataset/",transform=transform,)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset,test_dataset = random_split(dataset,[train_size,test_size])

In [71]:
train_dataloader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=64,shuffle=True)

In [72]:
epoch = 3
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cpu')

In [73]:
class BreastCancerPrediction(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,3,1)
    self.conv2 = nn.Conv2d(16,32,3,1)
    self.conv3 = nn.Conv2d(32,64,3,1)
    self.pooling = nn.MaxPool2d(2,2)
    self.flatten = nn.Flatten()
    self.linear1 = nn.Linear(64*26*26,256) # Corrected input size based on recalculation
    self.linear2 = nn.Linear(256,3)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = self.pooling(x)
    x = F.relu(self.conv2(x))
    x = self.pooling(x)
    x = F.relu(self.conv3(x))
    x = self.pooling(x)
    x = self.flatten(x)
    x = F.relu(self.linear1(x))
    x = self.linear2(x)
    return x

In [74]:
model = BreastCancerPrediction()

In [75]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [76]:
total_loss = 0
for epoch in range(epoch):
  print(f"Epoch: {epoch+1}")
  for batch,(x,y) in enumerate(train_dataloader):
    optimizer.zero_grad()
    pred = model(x)
    loss = loss_fn(pred,y)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Loss: {total_loss/len(train_dataloader)}")


Epoch: 1
Loss: 0.9762457996606827
Epoch: 2
Loss: 1.7971069008111953
Epoch: 3
Loss: 2.436761885881424


In [79]:
model.eval()
total_loss = 0
correct = 0
total_samples = 0

with torch.no_grad():
    for batch_idx, (data, targets) in enumerate(test_dataloader):
        preds = model(data)
        loss = loss_fn(preds, targets)

        total_loss += loss.item()
        correct += (preds.argmax(dim=1) == targets).sum().item()  # count correct
        total_samples += targets.size(0)  # total samples

    accuracy = 100 * correct / total_samples
    print(f"Loss: {total_loss/len(test_dataloader):.4f}  Accuracy: {accuracy:.2f}%")


Loss: 0.8252  Accuracy: 68.99%
